In [1]:
# get the data set
from word_mapping import word_mapping
from tag_mapping import tag_mapping
from utils import *
from check import check
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn
from HMM import *
import BiLSTM_CRF
from my_transformer import Transformer

# get the data set
from word_mapping import word_mapping
from tag_mapping import tag_mapping
from utils import *
from check import check
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn

cn_train_data, cn_valid_data = get_data_set('Chinese')
cn_TagMapping = tag_mapping('Chinese')
cn_WordMapping = word_mapping(cn_train_data)
cn_train_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_train_data.get_word_mapping(cn_WordMapping.encode_mapping)
cn_valid_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_valid_data.get_word_mapping(cn_WordMapping.encode_mapping)

en_train_data, en_valid_data = get_data_set('English')
en_TagMapping = tag_mapping('English')
en_WordMapping = word_mapping(en_train_data)
en_train_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_train_data.get_word_mapping(en_WordMapping.encode_mapping)
en_valid_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_valid_data.get_word_mapping(en_WordMapping.encode_mapping)

cn_tag_size = cn_TagMapping.num_tag
cn_vocab_size = cn_WordMapping.num_word

en_tag_size = en_TagMapping.num_tag
en_vocab_size = en_WordMapping.num_word

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
en_model = BiLSTM_CRF.BiLSTMCRF.load("BiLSTM_CRF/English.pth", device)

print('---------------- English Result ------------------')
language = 'English'
my_path = f'my_{language}_BiLSTM_CRF_result.txt'
result_file = open(my_path, 'w')
en_model.eval()
with torch.no_grad():
    for sentences, tags in en_valid_data:
        padded_sentences, sent_lengths = pad([sentences], en_vocab_size - 1, device)
        pred_tags = en_model.predict(padded_sentences, sent_lengths)
        sent = en_WordMapping.decode(sentences)
        pred_tags = en_TagMapping.decode(pred_tags[0])
        for token, pred_tag in zip(sent, pred_tags):
            result_file.write(f'{token} {pred_tag}\n')
        result_file.write('\n')
result_file.close()
gold_path = f"./NER/{language}/validation.txt"
check(language, gold_path, my_path)

del en_model

print('---------------- Chinese Result ------------------')
cn_model = BiLSTM_CRF.BiLSTMCRF.load("BiLSTM_CRF/Chinese.pth", device)
language = 'Chinese'
my_path = f'my_{language}_BiLSTM_CRF_result.txt'
result_file = open(my_path, 'w')
cn_model.eval()
with torch.no_grad():
    for sentences, tags in cn_valid_data:
        padded_sentences, sent_lengths = pad([sentences], cn_vocab_size - 1, device)
        pred_tags = cn_model.predict(padded_sentences, sent_lengths)
        sent = cn_WordMapping.decode(sentences)
        pred_tags = cn_TagMapping.decode(pred_tags[0])
        for token, pred_tag in zip(sent, pred_tags):
            result_file.write(f'{token} {pred_tag}\n')
        result_file.write('\n')
result_file.close()
gold_path = f"./NER/{language}/validation.txt"
check(language, gold_path, my_path)
del cn_model

---------------- English Result ------------------
              precision    recall  f1-score   support

       B-PER     0.9384    0.6122    0.7410      1617
       I-PER     0.9276    0.7318    0.8182      1156
       B-ORG     0.8621    0.6737    0.7563      1661
       I-ORG     0.8193    0.6898    0.7490       835
       B-LOC     0.8841    0.8231    0.8525      1668
       I-LOC     0.5040    0.7354    0.5981       257
      B-MISC     0.8263    0.7251    0.7724       702
      I-MISC     0.5625    0.6667    0.6102       216

   micro avg     0.8490    0.7083    0.7723      8112
   macro avg     0.7905    0.7072    0.7372      8112
weighted avg     0.8643    0.7083    0.7736      8112

---------------- Chinese Result ------------------
              precision    recall  f1-score   support

      B-NAME     0.9905    0.9286    0.9585       112
      M-NAME     0.9535    1.0000    0.9762        82
      E-NAME     0.9907    0.9554    0.9727       112
      S-NAME     0.0000    0.0